In [367]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [368]:
# Path to chromedriver
!which chromedriver

chromedriver not found


In [369]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/Users/jaimechusid/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver'}
browser = Browser('chrome', **executable_path)

In [370]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [371]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [372]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8805/moxie-could-help-future-rockets-launch-off-mars/" target="_self">MOXIE Could Help Future Rockets Launch Off Mars</a></div>

In [373]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'MOXIE Could Help Future Rockets Launch Off Mars'

In [374]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

### Featured Images


In [375]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [376]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [377]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [378]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [379]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA16220_hires.jpg'

In [380]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16220_hires.jpg'

In [381]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [382]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

### Mars Weather

In [383]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

In [384]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [385]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

<table class="mb_table" id="weather_observation" style="width:100%;">
 <thead>
  <tr>
   <th colspan="2" scope="col">
    Time
   </th>
   <th colspan="3" id="temperature_lbl" scope="col">
    Air Temperature (
    <span class="lbl_fahrenheit">
     °F
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_celsius fadeBlack">
     °C
    </span>
    )
   </th>
   <th colspan="4" id="windspeed_lbl" scope="col">
    Wind Speed (
    <span class="lbl_mph">
     mph
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_mps fadeBlack">
     m/s
    </span>
    )
   </th>
   <th colspan="3" id="pressure_lbl" scope="col">
    Pressure (Pa)
   </th>
  </tr>
 </thead>
 <tbody>
  <tr id="weather_top">
   <th class="sol" scope="row">
    Date
   </th>
   <th class="sol" scope="row">
    Sol
   </th>
   <td class="temperature max">
    Max.
   </td>
   <td class="temperature avg">
    Avg.
   </td>
   <td class="temperature min">
    Min.
   </td>
   <t

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [419]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [ ]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
hemisphere_soup = soup(html, 'html.parser')

base_url = 'https://astrogeology.usgs.gov'

hemi_section = hemisphere_soup.find_all('div', class_='item')
for hemi in hemi_section:
    hemisphere = {}
    hemi_title = hemi.find('h3').get_text()
    print(hemi_title)
    thumbnail = hemi.find('a', class_='itemLink product-item').get('href')
    print(thumbnail)
    browser.visit(base_url + thumbnail)
    html2 = browser.html
    hemi_soup2 = soup(html2, 'html.parser')
    hemi_img = hemi_soup2.find('img', class_='wide-image').get('src')
    print(hemi_img)
    hemisphere['title'] = hemi_title
    hemisphere['image_url'] = hemi_img
    hemisphere_image_urls.append(hemisphere)
    browser.back()
    

Cerberus Hemisphere Enhanced
/search/map/Mars/Viking/cerberus_enhanced
/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
/search/map/Mars/Viking/syrtis_major_enhanced


In [ ]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

In [ ]:
# 5. Quit the browser
browser.quit()